# 라이브러리

In [ ]:
! pip install konlpy 

In [67]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings(action='ignore')

# 사용자 정의 함수

In [51]:
# 텍스트 전처리
def text_preprocess(text):

  # 숫자, 기호, 영문 제거
  new_text = re.sub(r'[^가-힣]', ' ', text)

  # 불필요한 품사 제거
  okt = Okt()
  prepro_text = []
  for word, tag in okt.pos(new_text):
    if tag not in ['Josa', 'Suffix']:
      prepro_text.append(word)
  
  result = [word for word in prepro_text if word not in stop_words]

  #result = ' '.join(prepro_text)

  return result

# 데이터 로드 & 생성

In [40]:
# 파일 경로
old_path = '/content/drive/MyDrive/Colab Notebooks/블로그포스팅/쉬운성경/구약'
new_path = '/content/drive/MyDrive/Colab Notebooks/블로그포스팅/쉬운성경/신약'

old_testament = os.listdir(old_path)
new_testament = os.listdir(new_path)

# 순서대로 정렬이 안 돼 있어서 정렬
new_testament.sort()

In [41]:
# 구약
old_testament_df = pd.DataFrame(columns=['성경제목', '성경주소', '성경본문'])
for title in old_testament:
    each_old = old_path + '/' + title
    df = pd.read_table(each_old, encoding='cp949', names=['성경본문'])
    df['성경제목'] = title.split()[1].strip('.txt')
    df['성경본문'] = df['성경본문'].astype(str)
    df['성경주소'] = df['성경본문'].map(lambda x : x[ : x.find(' ')])
    df['성경본문'] = df['성경본문'].map(lambda x : x[x.find(' ')+1 :])
    df = df[['성경제목', '성경주소', '성경본문']]
    old_testament_df = pd.concat([old_testament_df, df], axis=0, ignore_index=True)

# 신약
new_testament_df = pd.DataFrame(columns=['성경제목', '성경주소', '성경본문'])
for title in new_testament:
    each_new = new_path + '/' + title
    df = pd.read_table(each_new, encoding='cp949', names=['성경본문'])
    df['성경제목'] = title.split()[1].strip('.txt')
    df['성경본문'] = df['성경본문'].astype(str)
    df['성경주소'] = df['성경본문'].map(lambda x : x[ : x.find(' ')])
    df['성경본문'] = df['성경본문'].map(lambda x : x[x.find(' ')+1 :])
    df = df[['성경제목', '성경주소', '성경본문']]
    new_testament_df = pd.concat([new_testament_df, df], axis=0, ignore_index=True)

# 성격 각 권별 텍스트 분석

In [42]:
# 구약 성경 리스트
old_testament_list = []
for title in old_testament:
  old_testament_list.append(title[title.find(' ') + 1:title.find('.')])

# 신양 성경 리스트
new_testament_list = []
for title in new_testament:
  new_testament_list.append(title.split()[1].strip('.txt'))

## 빈도 분석

In [ ]:
# 1번째 방법 : Counter 사용

# 창세기로 sample example
for title in old_testament_list[0:1]:
  
  # 각 성경별 본문 시리즈로 불러오기
  each_old_testament = old_testament_df['성경본문'][old_testament_df['성경제목']== title]

  # 불용어 사전
  stop_words = ['하셨습니다', '그런데', '그', '그러자', '있는', '아니었고', '없이', '했습니다', '것', '내', '이',
                '입니다', '너', '네', '저', '나', '해', '그리고']

  # 본문별 전처리
  each_old_testament = each_old_testament.map(text_preprocess)

  # 빈도분석 실행
  prepro_words = []
  for text in each_old_testament:
    prepro_words += text
    
  count = Counter(prepro_words)
  print(count.most_common())

  count_word = [word for word, vec in count.most_common(20)]
  count_vec = [vec for word, vec in count.most_common(20)]

  plt.figure()
  plt.barh(count_word[::-1], count_vec[::-1], color='tab:pink')
  plt.yticks(count_word[::-1])
  plt.show()